In [ ]:
# NBVAL_SKIP
import os
os.environ['SPS_HOME'] = '/mnt/storage/annalena_data/sps_fsps'
#os.environ['SPS_HOME'] = '/home/annalena/sps_fsps'
#os.environ['SPS_HOME'] = '/Users/annalena/Documents/GitHub/fsps'

# Fits files

In this notebook we show, how you can store your mock datacube in a fits file, which is the common format in which are observational data handled. We firtss create a mock IFU cube by running the RUBIX pipeline, store it then in a fits file and then lod the data from the fits file.

In [ ]:
# NBVAL_SKIP
import matplotlib.pyplot as plt
import os
from rubix.core.pipeline import RubixPipeline

# Define Illustris configuration
config_illustris = {
    "pipeline": {"name": "calc_ifu"},
    "logger": {"log_level": "DEBUG", "log_file_path": None, "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s"},
    "data": {
        "name": "NihaoHandler",
        "args": {
            "particle_type": ["stars", "gas"],
            "save_data_path": "data",
            "snapshot": "1024",
        },
        "load_galaxy_args": {"reuse": True, "id": "g8.26e11"},
        "subset": {"use_subset": True, "subset_size": 4000},
    },
    "simulation": {
        "name": "NIHAO",
        "args": {
            "path": '/mnt/storage/_data/nihao/nihao_classic/g8.26e11/g8.26e11.01024',
            "halo_path": '/mnt/storage/_data/nihao/nihao_classic/g8.26e11/g8.26e11.01024.z0.000.AHF_halos',
            "halo_id": 0,
        },
    },
    "output_path": "output",
    "telescope": {"name": "MUSE", "psf": {"name": "gaussian", "size": 5, "sigma": 0.6}, 
                  "lsf": {"sigma": 1.2}, "noise": {"signal_to_noise": 100, "noise_distribution": "normal"}},
    "cosmology": {"name": "PLANCK15"},
    "galaxy": {"dist_z": 0.3, "rotation": {"type": "edge-on"}},
    "ssp": {"template": {"name": "FSPS"}, #"Mastar_CB19_SLOG_1_5"},
            "dust": {
            "extinction_model": "Cardelli89", #"Gordon23", 
            "dust_to_gas_ratio": 0.01, # need to check Remyer's paper
            "dust_to_metals_ratio": 0.4, # do we need this ratio if we set the dust_to_gas_ratio?
            "dust_grain_density": 3.5, # g/cm^3 #check this value
            "Rv": 3.1,
        },
            },
}


# Run pipeline
pipe = RubixPipeline(config_illustris)
data = pipe.run()

In [ ]:
data.stars.datacube

## Convert luminosity to flux

In [ ]:
from rubix.spectra.ifu import convert_luminoisty_to_flux
from rubix.cosmology import PLANCK15

observation_lum_dist = PLANCK15.luminosity_distance_to_z(config_illustris["galaxy"]["dist_z"])
observation_z = config_illustris["galaxy"]["dist_z"]
pixel_size = 1.0
fluxcube = convert_luminoisty_to_flux(data.stars.datacube, observation_lum_dist, observation_z, pixel_size)
data.stars.datacube = fluxcube/1e-20


In [ ]:
data.stars.datacube

In [ ]:
fluxcube

In [ ]:
# NBVAL_SKIP
data.stars.spectra.shape

In [ ]:
# NBVAL_SKIP
data.stars.spectra.max()

In [ ]:
# NBVAL_SKIP
import numpy as np
plt.plot(np.linspace(1, 10, data.stars.spectra.shape[2]), data.stars.spectra[:,:750000,:].sum(axis=1)[1])

In [ ]:
#NBVAL_SKIP
datacube = data.stars.datacube

img = datacube.sum(axis=2)
plt.imshow(img, origin="lower")
plt.plot(12,12, 'ro')
plt.plot(17,12, 'x', color="blue")
plt.plot(7,12, 'x', color="orange")
plt.colorbar()
print(img.min(), img.max())

In [ ]:
# NBVAL_SKIP
wave = pipe.telescope.wave_seq
#plt.plot(wave, data.stars.datacube[12, 12, :], color="red", label="Spectrum")
#plt.vlines(4861.333, 0, 3000, color='r', label="Hbeta=4861.333A")
#plt.vlines(4861.333*1.1, 0, 3000, color='y', label="line obs=Hbeta*(1+z)")
plt.plot(wave, data.stars.datacube[7, 12, :], color="orange", label="Spectrum 7,12")
plt.plot(wave, data.stars.datacube[17, 12, :], color="blue", label="Spectrum 17,12")
#plt.xlim(5300, 5400)
plt.legend()

In [ ]:
# NBVAL_SKIP
wave = pipe.telescope.wave_seq
#plt.plot(wave, data.stars.datacube[12, 12, :], color="red", label="Spectrum")
plt.vlines(4861.333, 0, 10, color='r', label="Hbeta=4861.333A")
plt.vlines(4861.333*1.1, 0, 0.5, color='y', label="line obs=Hbeta*(1+z)")
plt.plot(wave, data.stars.datacube[14, 12, :], color="blue", label="Spectrum 2,12")
plt.plot(wave, data.stars.datacube[10, 12, :], color="orange", label="Spectrum 22,12")
plt.xlim(5300, 5400)
plt.legend()

In [ ]:
# NBVAL_SKIP
import matplotlib.pyplot as plt

# Plot a histogram of the velocities
plt.hist(data.stars.velocity[0,:,2], bins=30, edgecolor='black')
plt.xlabel('Velocity')
plt.ylabel('Frequency')
plt.title('Histogram of Star Velocities')
plt.show()

In [ ]:
# NBVAL_SKIP
import numpy as np
import matplotlib.pyplot as plt

# Assuming your data arrays are defined as follows:
pixel_assignment = np.asarray(np.squeeze(data.stars.pixel_assignment))
velocities = np.asarray(data.stars.velocity[0, :, 2])

# Compute the sum of velocities and count per pixel using np.bincount
sum_velocity = np.bincount(pixel_assignment, weights=velocities)
counts = np.bincount(pixel_assignment)

# Calculate mean velocity; note: division by zero is avoided if every pixel has at least one star.
mean_velocity = sum_velocity / counts


# If you know the pixel grid dimensions (for example, a square grid)
n_pixels = len(mean_velocity)
grid_size = int(np.sqrt(n_pixels))
if grid_size * grid_size != n_pixels:
    raise ValueError("The total number of pixels is not a perfect square; please specify the grid shape explicitly.")

# Reshape the mean_velocity into a 2D array for imshow
velocity_map = mean_velocity.reshape((grid_size, grid_size))
print(velocity_map[12,12])

print(velocity_map[17,12]-velocity_map[7,12])
# Plot the result
plt.figure(figsize=(6, 5))
plt.imshow(velocity_map, origin='lower', interpolation='nearest', cmap='seismic')
plt.colorbar(label='Mean Velocity')
plt.title('Mean Velocity per Pixel')
plt.xlabel('X pixel index')
plt.ylabel('Y pixel index')
plt.show()

In [ ]:
from rubix import config
import jax.numpy as jnp
SPEED_OF_LIGHT=config["constants"]["SPEED_OF_LIGHT"]
velocity = data.stars.velocity[0, :, 2]
doppler = jnp.exp(velocity/SPEED_OF_LIGHT)
print(doppler.min(), doppler.max())

# Store datacube in a fits file with header

In RUBIX we implemented a function that automaticly takes the relevant information from the config and writes it into the header. Then the header and data are stored in a fits file. All is done with the store_fits function from the rubix.core.fits module.

In [ ]:
#NBVAL_SKIP
from rubix.core.fits import store_fits

store_fits(config_illustris, data, "output/")

# Load datacube from fits file

We implemented a function to load a fits file. It is based on MPDAF, which is a package to handle MUSE IFU cubes. You can load your datacube by the following line and access all kind of information from the fitsfile.

In [ ]:
#NBVAL_SKIP
from rubix.core.fits import load_fits

cube = load_fits("output/IllustrisTNG_id11_snap99_stars_subsetTrue.fits") #if you use NIHAO, you have to insert the NIHAO fits file

In [ ]:
#NBVAL_SKIP
cube.shape

In [ ]:
#NBVAL_SKIP
cube.info()

In [ ]:
#NBVAL_SKIP
cube.primary_header

In [ ]:
#NBVAL_SKIP
import matplotlib.pyplot as plt

image1 = cube[0,:,:]

plt.figure()
image1.plot(colorbar='v', title = '$\lambda$ = %.1f (%s)' %(cube.wave.coord(1000), cube.wave.unit))
plt.show()